# Projeto 1 - Ciência dos Dados

Nome: Rodrigo Carvalho Monteiro de Barros

**Atenção:** Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [7]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

**Em `filename`, coloque o nome do seu arquivo de dados!**

In [13]:
import os

filename = 'HBO_max.xlsx'
if filename in os.listdir():
    print(f'Encontrei o arquivo {filename}, tudo certo para prosseguir com a prova!')
else:
    print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, será que você não baixou o arquivo?')

Encontrei o arquivo HBO_max.xlsx, tudo certo para prosseguir com a prova!


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [14]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,Relevancia
0,@hbomaxhelp yooo i’m already a hbo max subscri...,0
1,@hbo_max_context @noconviolence same vibe http...,0
2,@thecondomfish2 its on hbo max not hulu anymore,1
3,at&amp;t’s stankey: success of hbo max launch ...,1
4,"@dccomics heyyyyy, how about renewing it on hb...",1


In [15]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Relevancia
0,hbo max waking up every hour too see sapphics ...,1
1,at&amp;t ceo john stankey says it will launch ...,0
2,@eleni_argyro i havent! its on hbo max only so...,1
3,rt @marginalmarisa: me and my friends on my hb...,0
4,at&amp;t’s stankey: success of hbo max launch ...,0


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

Meu interesse no produto HBO max é saber a opinião e aceitação (ou não) dos usuarios ou possiveis usuarios, a fim de descobrir se o produto fez sucesso desde seu recente lançamento e se conseguirá lugar no crescente mercado de serviços de streeming.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**